In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Dense, Layer, Input,Dropout
from keras.models import Model,Sequential
from keras.layers import LayerNormalization

2024-06-16 14:53:04.650052: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-16 14:53:04.650141: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-16 14:53:04.780168: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pos encoding

In [2]:
def pos_encoding(max,input):
  d= input.shape[1]
  c=int(d/2)
  for pos in range(0,max):
    for i in range(0, c):
      pos[2*i] = np.sin(pos/(10000**( (2*i)/d)))
      pos[2*i + 1] = np.cos(pos/(10000**( (2*i)/d)))

  return pos_enc

AttributeError: 'int' object has no attribute 'shape'

Glove vector

In [3]:
pip install nltk

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import string

# Download necessary datasets
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
!wget --no-check-certificate https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


--2024-06-15 18:19:56--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: 'glove.6B.zip'

glove.6B.zip        100%[===================>] 822.24M  5.01MB/s    in 2m 39s  

2024-06-15 18:22:35 (5.18 MB/s) - 'glove.6B.zip' saved [862182613/862182613]



In [7]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [8]:
import os
import numpy as np

In [9]:
glove_dir = './'

embeddings_index = {} #initialize dictionary
f = open(os.path.join(glove_dir, 'glove.6B.300d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [10]:
# import zipfile
# import os

# # Path to the zip file
# zip_file_path = "/content/sobhins-classic.zip"  # Update with your zip file path

# # Directory to extract the files
# extract_dir = "/content/sample"  # Update with the directory where you want to extract the files

# # Check if the zip file exists
# if os.path.exists(zip_file_path):
#     # Create the extraction directory if it doesn't exist
#     os.makedirs(extract_dir, exist_ok=True)

#     # Extract the zip file
#     with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#         zip_ref.extractall(extract_dir)

#     print("Extraction completed successfully.")
# else:
#     print("Zip file not found.")


In [4]:
import pandas as pd

df = pd.read_csv('/kaggle/input/sobhins-classic/output2.csv')

In [5]:
df1 = df.dropna()
df2 = df1[['desc','genre']].copy()
df3 = df2.loc[:,:]
dfm = df3[df3['genre'].isin([' drama ',' documentary ',' comedy '])]

In [6]:
x_train = dfm['desc']
y_train = dfm['genre']

In [5]:
import nltk
import subprocess

# Download and unzip wordnet
try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')

# Now you can import the NLTK resources as usual
from nltk.corpus import wordnet

[nltk_data] Downloading package wordnet to /kaggle/working/...
[nltk_data]   Package wordnet is already up-to-date!
Archive:  /kaggle/working/corpora/wordnet.zip


replace /kaggle/working/corpora/wordnet/lexnames? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


In [12]:
word_index.size()

AttributeError: 'dict' object has no attribute 'size'

                                                  de  \
0  An der B 211 befindet sich in Loyermoor der so...   
1  Ich begrüße die Erklärung des Herrn Kommissar ...   
2  Das ist das Gegenteil von dem, was getan werde...   
3                                                  .   
4  The Ethnographical museum in Varna is in a hou...   

                                                  en  
0  Here the largest town of the district is locat...  
1  I should like, in passing, to pay tribute to t...  
2  That is the opposite of what should be done an...  
3                                                  .  
4  It was designed by the Viennese architect Rupp...  


In [7]:
import pandas as pd

In [167]:
df_clean = pd.read_csv('/kaggle/input/wmt-2014-english-german/wmt14_translate_de-en_train.csv',nrows=500000,
                 lineterminator='\n')

In [168]:
df_ger = df_clean['de']
df_eng = df_clean['en']

In [169]:
x_train = df_eng

In [171]:
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb


# Concatenate train and test data for preprocessing


# Function to preprocess text
def preprocess_text(text):
    """
    Preprocesses the input text by performing tokenization, converting to lowercase,
    removing punctuation, removing stopwords, and lemmatization.

    Args:
    text (str): The input text to be processed.

    Returns:
    List[str]: The list of preprocessed tokens.
    """
    # Tokenization
    tokens = word_tokenize(text)

    # Convert to Lowercase
    tokens = [word.lower() for word in tokens]

    # Remove Punctuation
    tokens = [word for word in tokens if word.isalpha()]

    #Remove Stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

#     # Lemmatization
#     lemmatizer = WordNetLemmatizer()
#     tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return tokens

# Preprocess all data
preprocessed_data = [preprocess_text(review) for review in x_train]


# Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts([' '.join(review) for review in preprocessed_data])


sequences = tokenizer.texts_to_sequences([' '.join(review) for review in preprocessed_data])
sequences = [arr + [-2] for arr in sequences]

word_index = tokenizer.word_index

# Pad sequences
max_length = 300
padded_dataE = pad_sequences(sequences, maxlen=max_length, padding='post')


In [123]:
last_elements[2]

991

In [178]:
len(word_index)

178057

In [180]:
word_index['<StarT>']=-1
word_index['<EnD>']=-2

In [179]:
X_train=np.array(padded_dataE)

In [181]:
x= np.zeros((500000,301))
for i in range(500000):
    x[i] = np.insert(X_train[i],0,-1)

In [182]:
x= x[:,:-1]

In [183]:
X_trainD=x

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
X_train = np.

In [40]:
x_trainG = df_clean['de']

In [46]:
x_trainG[0]

'An der B 211 befindet sich in Loyermoor der so genannte „Geest-Abbruch“, der eine Höhendifferenz von gut 30 Meter überbrückt.'

In [43]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [49]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_german_text(text):
    # Tokenization
    tokens = word_tokenize(text, language='german')
    
    # Lowercasing
    tokens = [token.lower() for token in tokens]
    
    # Remove punctuation and non-alphanumeric tokens
    tokens = [token for token in tokens if token.isalpha()]
    
    # Remove stopwords
    stop_words = set(stopwords.words('german'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatization (optional)
    # lemmatizer = WordNetLemmatizer()
    # tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Join tokens back into a sentence
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

preprocessed_dataG = [preprocess_german_text(review) for review in x_trainG]


# Tokenizer
tokenizerG = Tokenizer()
tokenizerG.fit_on_texts([' '.join(review) for review in preprocessed_dataG])


sequencesG = tokenizerG.texts_to_sequences([' '.join(review) for review in preprocessed_dataG])


word_indexG = tokenizerG.word_index

# Pad sequences
max_length = 300
padded_data = pad_sequences(sequencesG, maxlen=max_length, padding='post')


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [51]:
pip install german

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.7 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=b0cb4b73b8f80e4e3e736c8422a72d1f8ee3e6faff8d1302dec0632f04031183
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji
  Attempting uninstall: emoji
    Found existing installation: emoji 2.12.1
    Uninstalling emoji-2.12.1:
      Successfully uninstalled emoji-2.12.1
Note: you may need to restart the kernel to use updated packages.


In [52]:
from german import preprocess

def preprocess_german_text(text):
    
    preprocessed_text = preprocess(text, remove_stop=True)

    return preprocessed_text

# Preprocess all data
preprocessed_dataG = [preprocess_german_text(review) for review in x_trainG]


# Tokenizer
tokenizerG = Tokenizer()
tokenizerG.fit_on_texts([' '.join(review) for review in preprocessed_dataG])


sequencesG = tokenizerG.texts_to_sequences([' '.join(review) for review in preprocessed_dataG])


word_indexG = tokenizerG.word_index

# Pad sequences
max_length = 300
padded_data = pad_sequences(sequencesG, maxlen=max_length, padding='post')



  0%|          | 0/125 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
100%|██████████| 125/125 [00:01<00:00, 70.86it/s]
0it [00:00, ?it/s]


DockerException: Error while fetching server API version: Not supported URL scheme http+docker

In [14]:
# Example German sentence
german_sentence = "Ich habe den ganzen Tag Deutsch gelernt."

# Preprocess the German sentence
preprocessed_sentence = preprocess_german_text(german_sentence)
print("Preprocessed Sentence:", preprocessed_sentence)

Preprocessed Sentence: ich habe den ganzen tag deutsch gelernt .


In [17]:
padded_data.shape

(34156, 200)

In [18]:
embedding_dim = 300
num_words = len(word_index)+1
embedding_matrix = np.zeros((num_words, embedding_dim)) #create an array of zeros with word_num rows and embedding_dim columns
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < num_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [19]:
embedding_matrix.shape

(86447, 300)

In [15]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder



# Reshape y_train to a column vector (required by OneHotEncoder)
y_train =np.array( y_train)
y_train=y_train.reshape(-1, 1)

# Create an instance of OneHotEncoder
encoder = OneHotEncoder(sparse=False)

# Fit and transform y_train to one-hot encoded format
y_train_onehot = encoder.fit_transform(y_train)
print(y_train_onehot[0])



[0. 0. 1.]


/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


pos_enc + glove_emb

self attention mech

In [21]:
def softmax(matrix):
    # Subtract the max value from each row for numerical stability
    exp_matrix = np.exp(matrix - np.max(matrix, axis=1, keepdims=True))
    # Divide by the sum of exponentials for each row
    softmax_matrix = exp_matrix / np.sum(exp_matrix, axis=1, keepdims=True)
    return softmax_matrix

For weight matrices Q,K,V we are using Dense layers as they are trainable

In [22]:
x = np.random.rand(2,5,100) # u can send in batches
W = Dense(32,use_bias=False) # bias should not be there as these act only as matrices with weights even no activation funcs
Q = Dense(32,use_bias=False)
K = Dense(32,use_bias=False)
w= W(x)
Q(x)
K(x)

<tf.Tensor: shape=(2, 5, 32), dtype=float32, numpy=
array([[[ 1.3476851 ,  2.1903548 , -0.22787488, -0.06536621,
          0.14794889, -0.25170103,  0.26709646,  0.17225453,
         -0.13360104, -0.29655674,  1.0847025 ,  0.24041533,
          0.7929813 , -0.10451432, -1.1481694 ,  1.3591585 ,
          0.9238999 ,  0.40648383, -1.6882964 ,  0.967045  ,
          0.6605883 ,  0.22796515,  1.2884505 , -0.62256664,
          0.15587823,  0.98689127, -0.734223  , -0.43940696,
         -0.5137866 , -1.3471773 , -0.31607616, -0.39597237],
        [ 1.2252619 ,  2.2225633 , -0.9059501 , -0.48237148,
          0.7914054 , -0.09414245,  0.8324181 ,  0.7702593 ,
         -0.9307699 , -0.8684033 ,  0.41811052,  0.34334654,
          1.0489726 ,  0.9834802 , -1.631293  ,  0.7341723 ,
          0.38364398,  1.3851092 , -1.1220211 ,  1.0967622 ,
          0.0278936 ,  0.8224919 ,  1.1516116 , -0.05503222,
         -0.07610722,  0.6356843 , -0.78698903, -0.78363067,
          0.44623363, -1.340215 

In [23]:
V = Dense(32,use_bias=False) # bias should not be there as these act only as matrices with weights even no activation funcs
Q = Dense(32,use_bias=False)
K = Dense(32,use_bias=False)

def self_attention(x,Q,K,V):
  c=x.shape[0]
  k = K(x)
  q = Q(x)
  v = V(x)
  att = np.zeros((c,c))

  att = np.matmul(q,tf.transpose(k))
  att = att/np.sqrt(x.shape[1])
  att = softmax(att)

  # need to apply softmax while next step x as 2d mat
  w = np.matmul(att,v)
  return w

In [24]:

x = np.random.rand(2,20,10)
y =Dense(15)
z = y(x)
z1 = y(x)

#z2 = tf.matmul(z1,z,transpose_b=False)
# x3 = tf.concat([z, z], axis=1)
# x4= LayerNormalization()
# x=x4(x+x)
# x.shape

creating model


In [184]:
# Including the matrices and the function in single class

class SelfAttention(Layer):
    def __init__(self, embS):     #id --> input dim   embS --> matrice dim
        super(SelfAttention, self).__init__()
        self.embed_size = embS
        self.Q = Dense(embS, use_bias=False)
        self.K = Dense(embS, use_bias=False)
        self.V = Dense(embS, use_bias=False)

    def call(self, x):
        q = self.Q(x)
        k = self.K(x)
        v = self.V(x)

        att = tf.matmul(q, k, transpose_b=True) / tf.math.sqrt(tf.cast(self.embed_size, tf.float32))
        att_weights = tf.nn.softmax(att, axis=-1)
        output = tf.matmul(att_weights, v)

        return output

        """When subclassing tf.keras.Model, you need to define a call() method explicitly. This method specifies how inputs are processed through the model during both training and inference"""

In [185]:
# Define the model with self-attention

class AttentionModel(Model):
    def __init__(self, embed_size):
        super(AttentionModel, self).__init__()
        self.attention = SelfAttention(embed_size)

    def call(self, x):
        x = self.attention(x)
        x = tf.reduce_mean(x, axis=1)  # Pooling
        return x


Creating multihead attention

In [186]:
#inheriting selfattention model
class MultiHead(AttentionModel):
  def __init__(self,embS,id,heads=2):
    super(MultiHead,self).__init__(embS)
    self.att = [SelfAttention(embS) for _ in range(heads) ]  # multiple self att instances
    self.dense = Dense(id, use_bias = False)

  def call(self,x):
    outs = [ f(x) for f in self.att ]   # x--> inputs

    x3 = tf.concat(outs , axis=-1)
    x3 = self.dense(x3)
    return x3

    """Working for single sentence of any length  """



Add&Norm

In [187]:
from tensorflow import keras
from keras.layers import Input, Embedding, Dense, GlobalAveragePooling1D, MultiHeadAttention, LayerNormalization
from keras.models import Model
from keras.initializers import Constant

# Define the custom Positional Encoding Layer
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'position': self.pos_encoding.shape[0],
            'd_model': self.pos_encoding.shape[1],
        })
        return config

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(np.arange(position)[:, np.newaxis],
                                     np.arange(d_model)[np.newaxis, :],
                                     d_model)
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
        pos_encoding = angle_rads[np.newaxis, ...]
        return tf.cast(pos_encoding, dtype=tf.float32)

    def get_angles(self, pos, i, d_model):
        angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
        return pos * angle_rates

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]






In [29]:
def pos_encoding(max,input,d):
  pos_enc = np.zeros((max,d))
  c=int(d/2)
  for pos in range(0,max):
    for i in range(0, c):
      pos_enc[pos,2*i] = x[pos,2*i]+np.sin(pos/(10000**( (2*i)/d)))
      pos_enc[pos,2*i + 1] = x[pos,(2*i)+1]+np.cos(pos/(10000**( (2*i)/d)))

  return pos_enc

In [20]:
len(word_index)

86446

In [188]:
from keras.losses import CategoricalCrossentropy
from keras.layers import Embedding,GlobalAveragePooling1D,Dropout
from keras.optimizers import Adam

# Hyperparameters
embS = 16
batch_size = 100
heads = 2
max_length = 200
id=300
embedding_dim = id
num_words = len(word_index)+1

class encoder(Model):
  def __init__(self,max_length,num_words,embedding_dim,id,embS,heads):   #embS --> matrix embed size
    super(encoder,self).__init__()
    #self.inputs = tf.keras.Input(shape=(max_length,))
#     self.embeds = Embedding(input_dim=num_words ,
#                     output_dim=embedding_dim,
#                     weights=[embedding_matrix],
#                     input_length=max_length,
#                     trainable=False)
    self.le = Embedding(num_words,embedding_dim )
    self.pos_enc = PositionalEncoding(max_length, id)
    self.drop = Dropout(0.1)
    self.multiH = MultiHead(embS,id,heads)
    self.layer1 = LayerNormalization()
    self.dense1 = Dense(4*id,activation='relu')
    self.dense2 = Dense(id)
    self.layer2 = LayerNormalization()
    self.outputs = GlobalAveragePooling1D()
    #self.cc = Dense(3, activation='softmax')


  def call(self,x):
    #inputs = self.inputs(x)

    emb=  self.le(x)

    poser = self.pos_enc(emb)
    poser = self.drop(poser)
    multi =  self.multiH(poser)
    d1 = self.drop(poser + multi)
    AN1   =  self.layer1(d1)
    dense1 =  self.dense1(AN1)
    dense2 =  self.dense2(dense1)
    d2 = self.drop(AN1 + dense2)
    outputs1   =  self.layer2(d2)
    #outputM = self.outputs(outputs1)
    #outputcc = self.cc(outputM)

    return outputs1

# Final Output Layer


enc_model = encoder(max_length,num_words,embedding_dim,id,embS,heads)

adam_optimizer = Adam(beta_1=0.9, beta_2=0.98, epsilon=1e-9)
enc_model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])          ## Sparse categorical entropy FUCKS here



In [82]:
def preprocess_and_pad(text, max_length=200, use_stemming=False):
    # Tokenization
    tokens = word_tokenize(text)

    # Convert to Lowercase
    tokens = [word.lower() for word in tokens]

    # Remove Punctuation
    tokens = [word for word in tokens if word.isalpha()]



    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Stemming (optional)
    if use_stemming:
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(word) for word in tokens]

    # Convert tokens to integer indices
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts([' '.join(tokens)])
    sequences = tokenizer.texts_to_sequences([' '.join(tokens)])

    # Pad sequences
    max_length = 200
    padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

    return padded_sequences


stage-1 training #Done !!!! #

Decoder Starts

#1 Crossed multi

In [189]:
# Including the matrices and the function in single class

class CrossSelfAttention(Layer):
    def __init__(self, embS):     # q --> output of masked multi
        super(CrossSelfAttention, self).__init__()
        self.embed_size = embS
        self.K = Dense(embS, use_bias=False)
        self.V = Dense(embS, use_bias=False)
        self.Q = Dense(embS, use_bias=False)

    def call(self, x):                # 1--> masked multi output   0--> encoder output
        q = self.Q(x[1])
        k = self.K(x[0])
        v = self.V(x[0])

        att = tf.matmul(q, k, transpose_b=True) / tf.math.sqrt(tf.cast(self.embed_size, tf.float32))
        att_weights = tf.nn.softmax(att, axis=-1)
        output = tf.matmul(att_weights, v)

        return output

        """When subclassing tf.keras.Model, you need to define a call() method explicitly. This method specifies how inputs are processed through the model during both training and inference"""

In [190]:
class CrossedMulti(CrossSelfAttention):
  def __init__(self,embS,id,heads=2):      #q-->encoder output     id--> input word dimension   embS --> embed dimension
    super(CrossedMulti,self).__init__(embS)
    self.att = [CrossSelfAttention(embS) for _ in range(heads) ]  # multiple self att instances
    self.dense = Dense(id, use_bias = False)

  def call(self,x):
    outs = [ f(x) for f in self.att ]   # x--> inputs

    x3 = tf.concat(outs , axis=-1)
    x3 = self.dense(x3)
    return x3


#2 Masked multi

In [191]:
import tensorflow as tf
from keras.layers import Dense, Layer
import numpy as np

class MaskedSelfAttention(Layer):
    def __init__(self, emb, nw, wd):  # wd is ideally 64
        super(MaskedSelfAttention, self).__init__()
        self.embed_size = emb
        self.nw = nw
        self.wd = wd
        self.K = Dense(emb, use_bias=False)
        self.V = Dense(emb, use_bias=False)
        self.Q = Dense(emb, use_bias=False)

    def masker(self):
        mask = np.zeros((self.nw, self.nw))
        for i in range(self.nw):
            for j in range(i + 1, self.nw):
                mask[i, j] = -np.inf
        return mask

    def call(self, x):
        q = self.Q(x)
        k = self.K(x)
        v = self.V(x)
        M = self.masker()
        att = tf.matmul(q, k, transpose_b=True) / tf.math.sqrt(tf.cast(self.embed_size, tf.float32))
        M = tf.cast(tf.convert_to_tensor(M),dtype = tf.float32)
        att = att + M
        att_weights = tf.nn.softmax(att, axis=-1)
        output = tf.matmul(att_weights, v)
        return output


In [192]:
class Maskedmulti(MaskedSelfAttention):
    def __init__(self, emb, nw, wd, heads=2):
        super(Maskedmulti, self).__init__(emb, nw, wd)
        self.heads = heads
        self.att = [MaskedSelfAttention(emb, nw, wd) for _ in range(heads)]
        self.dense = Dense(wd, use_bias=False)

    def call(self, x):
        outs = [head(x) for head in self.att]
        x3 = tf.concat(outs, axis=-1)
        x3 = self.dense(x3)
        return x3


Time to pack everything

In [193]:
import tensorflow as tf
from keras.layers import Embedding, Dense, LayerNormalization, GlobalAveragePooling1D

class decoderModel(tf.keras.Model):
    def __init__(self ,num_words, embedding_dim,max_length, Dembd, nw, wd, heads): # embd --> matrixD in transformers wd--> wordD  nw -->max_length
        super(decoderModel, self).__init__()
#         self.embedding_layer = Embedding(input_dim=num_words,
#                                          output_dim=embedding_dim,
#                                          weights=[embedding_matrix],
#                                          input_length=max_length,
#                                          trainable=False)
        self.le = Embedding(num_words,embedding_dim)
        self.positional_encoding = PositionalEncoding(position=nw, d_model=wd)
        self.maskedmulti_layer = Maskedmulti(Dembd,nw, wd, heads)
        self.crossedmulti_layer = CrossedMulti(Dembd, wd, heads)
        self.layer_norm1 = LayerNormalization()
        self.layer_norm2 = LayerNormalization()
        self.dense1 = Dense(4*wd, activation='relu')
        self.dense2 = Dense(wd, activation='relu')
        self.global_avg_pooling = GlobalAveragePooling1D()
        self.dense3 = Dense(1,activation = 'linear')
        self.drop = Dropout(0.1)

    def call(self, inputs):
        emb = self.le(inputs[0])
        poser = self.positional_encoding(emb)
        maskmulti = self.maskedmulti_layer(poser)
        d1 = self.drop(emb + maskmulti)
        SL1 = self.layer_norm1(d1)
        cross = self.crossedmulti_layer([SL1,inputs[1]])
        d2 = self.drop(SL1 + cross)
        SL2 = self.layer_norm2(d2)
        dense1_out = self.dense1(SL2)
        dense2_out = self.dense2(dense1_out)
        d3 = self.drop(SL2 + dense2_out)
        SL3 = self.layer_norm2(d3)
        #outputM = self.global_avg_pooling(SL3)
        output = self.dense3(SL3)
        output = tf.transpose(output, perm=[0,2,1])

        return output


combining two classes into transformer

In [194]:
class Transformer(Model):
  def __init__(self,encoder,decoder):
    super(Transformer,self).__init__()
    super(Transformer).__init__()
    self.enc = encoder
    self.dec = decoder

  def call(self,inputs):
    enc1 = self.enc(inputs[0])
    dec1 = self.dec([inputs[1],enc1])

    return dec1


In [195]:
max_length=len(word_index)+1

In [140]:
last_elements=np.array(last_elements,dtype=np.int32)

In [135]:
last_elements.shape

(5000,)

In [1]:
embS = 64
batch_size = 100
heads =4
nw = 300
wd =512
id = 512
embedding_dim=512
num_words = nw

enc = encoder(max_length,num_words,embedding_dim,id,embS,heads)
dec1 = decoderModel(num_words, embedding_dim,max_length,embS , nw , id , heads )

model = Transformer(enc,dec1)

adam_optimizer = Adam(beta_1=0.9, beta_2=0.98, epsilon=1e-9)
model.compile(optimizer=adam_optimizer, loss='mse', metrics=['accuracy'])



model.fit([X_train,X_trainD],X_train,epochs =100000)


NameError: name 'encoder' is not defined

In [154]:
y = model.predict([X_train[:3],X_trainD[:3]])

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [155]:
y.shape

(3, 1, 300)

In [156]:
X_train.shape

(5000, 300)

In [55]:
model.save_weights('main1q.weights.h5')

In [197]:
model.summary()

Model: "transformer_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder_19 (encoder)            │ ?                      │     2,779,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder_model_17 (decoderModel) │ ?                      │     3,304,449 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,252,293 (69.63 MB)

 Trainable params: 6,084,097 (23.21 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 12,168,196 (46.42 MB)

In [57]:
X_train.shape

(34156, 200)

In [76]:
from keras.models import Model

# Remove the output layer from the original model
inputs = Input(shape=(200,))

op = model.enc(inputs)
op1 = GlobalAveragePooling1D()(op)

model2 = Model(inputs=inputs, outputs=op1)


In [102]:
t1= 'Angle is danger'
t2= 'Devil'
x1 = preprocess_and_pad(t1)
x2 = preprocess_and_pad(t2)
y1= model2.predict(x1)
y2 = model2.predict(x2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


In [103]:
y1-y2

array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00, -1.4901161e-08,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00, -3.7252903e-09,  0.0000000e+00,
         0.0000000e+00,  1.4901161e-08,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.00000